In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
np.set_printoptions(suppress=True)#不输出科学计数法
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus']=False #显示中文和正负号
%config InlineBackend.figure_format='svg' #增加清晰度
from pandas import DataFrame,Series
#设定绘图风格
plt.style.use('ggplot')

In [2]:
df=pd.read_csv(r'D:\notebook\data20200206\contb_02.csv')
df.head()

,cand_nm,contbr_nm,contbr_st,contbr_employer,contbr_occupation,contb_receipt_amt,contb_receipt_dt
0,"Obama, Barack","THOMAS, MAMIE ETHEL",NY,RETIRED,RETIRED,200.0,30-SEP-11
1,"Obama, Barack","PIZZI, JULIA",NY,DEWEY & LEBOEUF,ATTORNEY,100.0,14-SEP-11
2,"Obama, Barack","LEHRER, RANDY",NY,SELF-EMPLOYED,PSYCHOTHERAPIST,10.0,04-JUL-11
3,"Obama, Barack","MARKS, ELIZABETH",NY,MATTHEW MARKS GALLERY,DIRECTOR ART GALLERY,150.0,30-SEP-11
4,"Obama, Barack","GELB, DANIEL",NY,SOUTH BEACH PSYCHIATRIC CENTER,SOCIAL WORKER,150.0,04-AUG-11


In [8]:
c2=pd.read_csv(r'D:\notebook\data20200206\contb_01.csv')
c2.head()

,cand_nm,contbr_nm,contbr_st,contbr_employer,contbr_occupation,contb_receipt_amt,contb_receipt_dt
0,"Bachmann, Michelle","HARVEY, WILLIAM",AL,RETIRED,RETIRED,250.0,20-JUN-11
1,"Bachmann, Michelle","HARVEY, WILLIAM",AL,RETIRED,RETIRED,50.0,23-JUN-11
2,"Bachmann, Michelle","SMITH, LANIER",AL,INFORMATION REQUESTED,INFORMATION REQUESTED,250.0,05-JUL-11
3,"Bachmann, Michelle","BLEVINS, DARONDA",AR,NONE,RETIRED,250.0,01-AUG-11
4,"Bachmann, Michelle","WARDENBURG, HAROLD",AR,NONE,RETIRED,300.0,20-JUN-11


In [9]:
c3=pd.read_csv(r'D:\notebook\data20200206\contb_03.csv')
c3.head()

,cand_nm,contbr_nm,contbr_st,contbr_employer,contbr_occupation,contb_receipt_amt,contb_receipt_dt
0,"Perry, Rick","NORRIS, BOBBY",TX,SELF,REALTOR/ RANCHER,1000.0,14-DEC-11
1,"Perry, Rick","HUNT, CAROLINE",TX,HOMEMAKER,HOMEMAKER,2500.0,14-DEC-11
2,"Perry, Rick","ROBINOWITZ, DALE",TX,SELF-EMPLOYED,DENTIST,2500.0,14-DEC-11
3,"Perry, Rick","ZUNIGA, DAVID & SYLVIA",TX,DAVID M. ZUNIGA PE,OWNER,250.0,14-DEC-11
4,"Perry, Rick","FOSTER, GLENNA",TX,SELF,INVESTOR/ CONSULTANT,1500.0,14-DEC-11


## 1 数据的预处理（nan，时间序列，基本的统计描述）

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500001 entries, 0 to 500000
Data columns (total 7 columns):
cand_nm              500001 non-null object
contbr_nm            500001 non-null object
contbr_st            500001 non-null object
contbr_employer      490448 non-null object
contbr_occupation    495774 non-null object
contb_receipt_amt    500001 non-null float64
contb_receipt_dt     500001 non-null object
dtypes: float64(1), object(6)
memory usage: 26.7+ MB


### 2只选取这些内容进行分析
#cand_nm竞选人
  #contbr_nm 献金人
  #contbr_st 献金人州
  #contbr_employer捐赠人所在公司
  #contbr_occupation捐赠人职业
  #contb_receipt_amt捐赠数额
  #contb_receipt_dt捐赠日期

In [9]:
# pd.to_datetime(df['contb_receipt_dt']) #这里其实可以直接将object转换成时间格式 但是为了后续的计算这个暂时不用

In [4]:
df=df[['cand_nm','contbr_nm','contbr_st','contbr_employer',
      'contbr_occupation','contb_receipt_amt','contb_receipt_dt']] #切片以传入列表方式将题目需要的列给提取出来
df.head()

,cand_nm,contbr_nm,contbr_st,contbr_employer,contbr_occupation,contb_receipt_amt,contb_receipt_dt
0,"Obama, Barack","THOMAS, MAMIE ETHEL",NY,RETIRED,RETIRED,200.0,30-SEP-11
1,"Obama, Barack","PIZZI, JULIA",NY,DEWEY & LEBOEUF,ATTORNEY,100.0,14-SEP-11
2,"Obama, Barack","LEHRER, RANDY",NY,SELF-EMPLOYED,PSYCHOTHERAPIST,10.0,04-JUL-11
3,"Obama, Barack","MARKS, ELIZABETH",NY,MATTHEW MARKS GALLERY,DIRECTOR ART GALLERY,150.0,30-SEP-11
4,"Obama, Barack","GELB, DANIEL",NY,SOUTH BEACH PSYCHIATRIC CENTER,SOCIAL WORKER,150.0,04-AUG-11


3#用用统计学数据 快速描述数据类型种类describe（）

In [5]:
df.describe()

,contb_receipt_amt
count,500001.000000
mean,240.417336
std,560.771601
min,-5115.000000
25%,35.000000
50%,100.000000
75%,250.000000
max,25000.000000


4#因为保密原因，要把空值填充尾 NOT PROVIDED

In [9]:
df.fillna(value='NOT PROVIDED',inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500001 entries, 0 to 500000
Data columns (total 7 columns):
cand_nm              500001 non-null object
contbr_nm            500001 non-null object
contbr_st            500001 non-null object
contbr_employer      500001 non-null object
contbr_occupation    500001 non-null object
contb_receipt_amt    500001 non-null float64
contb_receipt_dt     500001 non-null object
dtypes: float64(1), object(6)
memory usage: 26.7+ MB


5#异常值处理，对捐赠金额小于零的数据删除处理

In [10]:
df['contb_receipt_amt']<=0

0         False
1         False
2         False
3         False
4         False
5         False
6         False
7         False
8         False
9         False
10        False
11        False
12        False
13        False
14        False
15        False
16        False
17        False
18        False
19        False
20        False
21        False
22        False
23        False
24        False
25        False
26        False
27        False
28        False
29        False
          ...  
499971    False
499972    False
499973    False
499974    False
499975    False
499976    False
499977    False
499978    False
499979    False
499980    False
499981     True
499982    False
499983    False
499984    False
499985    False
499986    False
499987    False
499988    False
499989    False
499990    False
499991    False
499992    False
499993     True
499994    False
499995    False
499996    False
499997    False
499998    False
499999    False
500000    False
Name: contb_receipt_amt,

In [11]:
df=df.loc[~(df['contb_receipt_amt']<=0)] #取反

6#新建一列，各个候选人所在的党派

In [6]:
#共有多少不同的候选人
df['cand_nm'].unique()

array(['Obama, Barack', "Roemer, Charles E. 'Buddy' III",
       'Pawlenty, Timothy', 'Johnson, Gary Earl', 'Paul, Ron',
       'Santorum, Rick', 'Cain, Herman', 'Gingrich, Newt',
       'Bachmann, Michelle', 'McCotter, Thaddeus G', 'Huntsman, Jon',
       'Perry, Rick'], dtype=object)

In [7]:
#用到映射
parties = {
  'Bachmann, Michelle': 'Republican',
  'Romney, Mitt': 'Republican',
  'Obama, Barack': 'Democrat',
  "Roemer, Charles E. 'Buddy' III": 'Reform',
  'Pawlenty, Timothy': 'Republican',
  'Johnson, Gary Earl': 'Libertarian',
  'Paul, Ron': 'Republican',
  'Santorum, Rick': 'Republican',
  'Cain, Herman': 'Republican',
  'Gingrich, Newt': 'Republican',
  'McCotter, Thaddeus G': 'Republican',
  'Huntsman, Jon': 'Republican',
  'Perry, Rick': 'Republican'           
 }

In [8]:
df['cand_nm'].map(parties)

0           Democrat
1           Democrat
2           Democrat
3           Democrat
4           Democrat
5           Democrat
6           Democrat
7           Democrat
8           Democrat
9           Democrat
10          Democrat
11          Democrat
12          Democrat
13          Democrat
14          Democrat
15          Democrat
16          Democrat
17          Democrat
18          Democrat
19          Democrat
20          Democrat
21          Democrat
22          Democrat
23          Democrat
24          Democrat
25          Democrat
26          Democrat
27          Democrat
28          Democrat
29          Democrat
             ...    
499971    Republican
499972    Republican
499973    Republican
499974    Republican
499975    Republican
499976    Republican
499977    Republican
499978    Republican
499979    Republican
499980    Republican
499981    Republican
499982    Republican
499983    Republican
499984    Republican
499985    Republican
499986    Republican
499987    Rep

In [9]:
df['parties']=df['cand_nm'].map(parties)

In [16]:
df.head()

,cand_nm,contbr_nm,contbr_st,contbr_employer,contbr_occupation,contb_receipt_amt,contb_receipt_dt,parties
0,"Obama, Barack","THOMAS, MAMIE ETHEL",NY,RETIRED,RETIRED,200.0,30-SEP-11,Democrat
1,"Obama, Barack","PIZZI, JULIA",NY,DEWEY & LEBOEUF,ATTORNEY,100.0,14-SEP-11,Democrat
2,"Obama, Barack","LEHRER, RANDY",NY,SELF-EMPLOYED,PSYCHOTHERAPIST,10.0,04-JUL-11,Democrat
3,"Obama, Barack","MARKS, ELIZABETH",NY,MATTHEW MARKS GALLERY,DIRECTOR ART GALLERY,150.0,30-SEP-11,Democrat
4,"Obama, Barack","GELB, DANIEL",NY,SOUTH BEACH PSYCHIATRIC CENTER,SOCIAL WORKER,150.0,04-AUG-11,Democrat


7#查看parties这一列中有哪些不同的元素，并统计parties中元素出现的次数

In [10]:
df['parties'].unique()

array(['Democrat', 'Reform', 'Republican', 'Libertarian'], dtype=object)

In [18]:
df['parties'].count() #发现不对

495156

In [11]:
df['parties'].value_counts()  #value_counts()就是用来统计series里面不同项目的个数汇总

Republican     291461
Democrat       201386
Reform           5920
Libertarian      1234
Name: parties, dtype: int64

#8查看各个党派收到的政治献金的总数

In [19]:
#先用透视表来表达分组
pd.pivot_table(df,values='contb_receipt_amt',index='parties',aggfunc='sum')

,contb_receipt_amt
parties,
Democrat,4.170916e+07
Libertarian,5.669616e+05
Reform,3.674575e+05
Republican,7.756533e+07


In [20]:
#groupby的方法
df.groupby(by='parties')['contb_receipt_amt'].sum().apply(lambda x: '%.2f' % x)

parties
Democrat       41709162.18
Libertarian      566961.60
Reform           367457.54
Republican     77565327.20
Name: contb_receipt_amt, dtype: object

#9查看每天每个党派收到的钱钱的总数

In [21]:
df.groupby(by=['contb_receipt_dt','parties'])['contb_receipt_amt'].sum()

contb_receipt_dt  parties    
01-APR-11         Reform              50.00
                  Republican       12635.00
01-APR-12         Democrat         65569.22
                  Libertarian        625.00
                  Republican       53414.32
01-AUG-11         Democrat         54443.00
                  Libertarian       1000.00
                  Reform            1847.00
                  Republican      112621.46
01-DEC-11         Democrat        254991.97
                  Libertarian        725.00
                  Reform             875.00
                  Republican      246984.28
01-FEB-11         Republican         250.00
01-FEB-12         Democrat        220375.74
                  Libertarian        475.00
                  Reform            2695.00
                  Republican      182476.92
01-JAN-11         Republican        8600.00
01-JAN-12         Democrat         31090.98
                  Libertarian       2275.00
                  Reform             515.00
  

In [23]:
#先用透视表来表达分组
pd.pivot_table(df,values='contb_receipt_amt',index=['contb_receipt_dt','parties'],aggfunc='sum')

contb_receipt_amt
contb_receipt_dt parties                       
01-APR-11        Reform                   50.00
                 Republican            12635.00
01-APR-12        Democrat              65569.22
                 Libertarian             625.00
                 Republican            56939.32
01-AUG-11        Democrat              59345.00
                 Libertarian            1000.00
                 Reform                 1847.00
                 Republican           127621.46
01-DEC-11        Democrat             255241.97
                 Libertarian             725.00
                 Reform                  875.00
                 Republican           254524.28
01-FEB-11        Republican              250.00
01-FEB-12        Democrat             223275.74
                 Libertarian             475.00
                 Reform                 2695.00
                 Republican           186941.92
01-JAN-11        Republican             8600.00
01-JAN-12        Democrat              34840.98
                 Libertarian            2275.00
                 Reform                  515.00
                 Republican            62828.44
01-JUL-11        Democrat              21716.00
                 Libertarian            2000.00
                 Reform                  100.00
                 Republican            71985.74
01-JUN-11        Democrat              63061.00
                 Libertarian             500.00
                 Reform                   50.00
...                                         ...
30-SEP-11        Republican          3148994.62
31-AUG-11        Democrat             165634.00
                 Libertarian           10750.00
                 Reform                  450.00
                 Republican           979999.90
31-DEC-11        Democrat             535324.96
                 Reform                  695.00
                 Republican           453750.67
31-JAN-11        Republican             6000.00
31-JAN-12        Democrat             279143.72
                 Libertarian            1000.00
                 Reform                  150.00
                 Republican           636193.61
31-JUL-11        Democrat               7109.00
                 Reform                 1066.00
                 Republican             9781.02
31-MAR-11        Reform                  200.00
                 Republican            74575.00
31-MAR-12        Democrat             807804.96
                 Libertarian             250.00
                 Reform                  242.00
                 Republican           438907.56
31-MAY-11        Democrat              97509.66
                 Libertarian             250.00
                 Reform                  100.00
                 Republican           115659.80
31-OCT-11        Democrat              41834.12
                 Libertarian            4250.00
                 Reform                 3205.00
                 Republican           300105.93

[1549 rows x 1 columns]

10#将原始数据中的时间格式改成yy-mm-dd

In [23]:
months = {'JAN' : 1, 'FEB' : 2, 'MAR' : 3, 'APR' : 4, 'MAY' : 5, 'JUN' : 6,
          'JUL' : 7, 'AUG' : 8, 'SEP' : 9, 'OCT': 10, 'NOV': 11, 'DEC' : 12}

In [24]:
months['SEP']

9

In [25]:
def transform_date(s):
    day,month,year=s.split('-')   #因为显示的数据是数字和英文，不能改变列细节的情况下只能想到拆分
    month = months[month]        #month里面是sep这些英文，带入字典会返回相应的int数字对应相应的月份
    return '20'+year+'-'+str(month)+'-'+day #因为字典里的数字是int 而初始时间序列里的数据是object，需要改格式
#return 返回拼接的字符串

In [34]:
df['contb_receipt_dt'].map(transform_date)

0          2011-9-30
1          2011-9-14
2          2011-7-04
3          2011-9-30
4          2011-8-04
5          2011-9-21
6          2011-9-26
7          2011-9-28
8          2011-8-04
9          2011-9-27
10         2011-7-25
11         2011-9-23
12         2011-9-27
13         2011-8-04
14         2011-9-07
15         2011-9-27
16         2011-8-29
17         2011-9-09
18         2011-9-08
19         2011-9-05
20         2011-9-16
21         2011-8-05
22         2011-7-26
23         2011-8-12
24         2011-7-13
25         2011-9-30
26         2011-9-16
27         2011-9-27
28         2011-8-04
29         2011-8-22
             ...    
499971    2011-12-20
499972    2011-12-20
499973    2011-12-20
499974    2011-12-27
499975    2011-11-15
499976    2011-11-15
499977    2011-11-15
499978    2011-12-20
499979    2011-12-15
499980    2011-12-15
499981    2011-12-15
499982    2011-12-15
499983    2011-12-15
499984    2011-12-15
499985    2011-12-15
499986    2011-12-15
499987    201

#最后将这行覆盖，就能得到具体的时间数据 用df看就行了
#df['contb_receipt_dt']=df['contb_receipt_dt'].map(transform_date)

11#查看老兵这个职业支持谁Disabled Veteran


In [35]:
df['contbr_occupation']=='DISABLED VETERAN'

0         False
1         False
2         False
3         False
4         False
5         False
6         False
7         False
8         False
9         False
10        False
11        False
12        False
13        False
14        False
15        False
16        False
17        False
18        False
19        False
20        False
21        False
22        False
23        False
24        False
25        False
26        False
27        False
28        False
29        False
          ...  
499971    False
499972    False
499973    False
499974    False
499975    False
499976    False
499977    False
499978    False
499979    False
499980    False
499981    False
499982    False
499983    False
499984    False
499985    False
499986    False
499987    False
499988    False
499989    False
499990    False
499991    False
499992    False
499993    False
499994    False
499995    False
499996    False
499997    False
499998    False
499999    False
500000    False
Name: contbr_occupation,

In [39]:
df5=df.loc[df['contbr_occupation']=='DISABLED VETERAN']

In [40]:
df5.groupby(by='cand_nm').aggregate({'contb_receipt_amt':'sum'}).sort_values('contb_receipt_amt',ascending=False)

,contb_receipt_amt
cand_nm,
"Paul, Ron",4439.1
"Obama, Barack",1585.0
"Santorum, Rick",550.0
"Cain, Herman",300.0
"Gingrich, Newt",250.0
"Roemer, Charles E. 'Buddy' III",100.0


In [42]:
df5.groupby(by='cand_nm')['contb_receipt_amt'].sum().sort_values(ascending=False)

cand_nm
Paul, Ron                         4439.1
Obama, Barack                     1585.0
Santorum, Rick                     550.0
Cain, Herman                       300.0
Gingrich, Newt                     250.0
Roemer, Charles E. 'Buddy' III     100.0
Name: contb_receipt_amt, dtype: float64

#11找出各个候选人，的捐赠者捐赠金额最大的职业，以及捐赠数额

In [43]:
s=df.groupby(by=['cand_nm']).aggregate({'contb_receipt_amt':'max'})#选出候选人和接受献金的最大金额的合并表
s                                                               #然后塞入原表 关联一对一对应的职业从而展示

,contb_receipt_amt
cand_nm,
"Bachmann, Michelle",5000.0
"Cain, Herman",10000.0
"Gingrich, Newt",5100.0
"Huntsman, Jon",5000.0
"Johnson, Gary Earl",2500.0
"McCotter, Thaddeus G",4000.0
"Obama, Barack",7000.0
"Paul, Ron",25000.0
"Pawlenty, Timothy",10000.0


In [40]:
for i in range(s.size):#s.size就是返回矩阵中元素的个数，等于是将矩阵中的数据一一遍历
    ex='cand_nm=="%s" & contb_receipt_amt==%s'%(s.index[i],s.values[i])
# cand中的值等于%s中的值且等于amt列中的相对应的值“”这里面的相对应的函数%表示就是s.index[i]
#s矩阵中的索引i遍历，因为i in range（s.size）,因为用的是同一个i所以数据是一一对应的。这样带入df的ex才能一致
#比如 in range（5）就是在 【1,2,3,4,5】中的数据，一个个取出来
    print(ex)
    #display(df.query(ex))

cand_nm=="Bachmann, Michelle" & contb_receipt_amt==[5000.]
cand_nm=="Cain, Herman" & contb_receipt_amt==[10000.]
cand_nm=="Gingrich, Newt" & contb_receipt_amt==[5100.]
cand_nm=="Huntsman, Jon" & contb_receipt_amt==[5000.]
cand_nm=="Johnson, Gary Earl" & contb_receipt_amt==[2500.]
cand_nm=="McCotter, Thaddeus G" & contb_receipt_amt==[4000.]
cand_nm=="Obama, Barack" & contb_receipt_amt==[7000.]
cand_nm=="Paul, Ron" & contb_receipt_amt==[25000.]
cand_nm=="Pawlenty, Timothy" & contb_receipt_amt==[10000.]
cand_nm=="Perry, Rick" & contb_receipt_amt==[10000.]
cand_nm=="Roemer, Charles E. 'Buddy' III" & contb_receipt_amt==[200.]
cand_nm=="Santorum, Rick" & contb_receipt_amt==[10000.]
